## The objectives are as follows:


- Respond to Business Requirement 2:

    - There is a client who wants to know whether a given image of a cherry leaf is healthy or if it has powdery mildew.



## Inputs


    Following sets:

* Train

* Validation

* Test



## Outputs


- Images distribution plot in train, validation, and test set.

- Increasing data diversity by enhancing images.

- Adjusting label predictions during inference by modifying class indices.

- Machine learning model creation and training.

- The trained model should be preserved (saved).

- The learning curve is plotted to visualize the performance of the model.

- Evaluating the model's performance.

- Making predictions on a randomly selected image file.

In [ ]:
import os
current_dir = os.getcwd()
current_dir



In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import joblib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import random
import math



In [ ]:
my_data_dir = 'inputs/datasets/cherry-leaves' 
my_data_dir = '/workspace/project5/inputs/datasets/cherry-leaves/'

train_path = my_data_dir + '/train'
validation_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'
my_data_dir

version = 'v2'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
  print('There is a version is already available, please create a new version.')
  pass
else:
  os.makedirs(name=file_path)


In [ ]:
labels = os.listdir(train_path)

print(f"The project labels are: {labels}")

version = 'v2'
image_shape = joblib.load(filename=f"outputs/{version}/images_shapes.pkl")
image_shape

### Retrieve the counts of images in the train, validation, and test datasets, and generate a plot displaying the distribution of images across these datasets.

In [ ]:
image_counts = []
for folder in ['train', 'validation', 'test']:
    counts = [len(os.listdir(f"{my_data_dir}/{folder}/{label}")) for label in labels]
    image_counts.extend(zip([folder] * len(labels), labels, counts))
    print('\n'.join([f"* {folder} - {label}: {count} images" for label, count in zip(labels, counts)]))

image_counts_summary = pd.DataFrame(image_counts, columns=['Set', 'Label', 'Frequency'])
labels_with_folders = [f"{folder}: {label}" for folder, label in zip(image_counts_summary['Set'], image_counts_summary['Label'])]

custom_colors = ["#4287f5", "#f54242", "#42f56a", "#f5ad42", "#7a42f5", "#42f5e8"]

plt.pie(
    image_counts_summary['Frequency'],
    labels=labels_with_folders,
    autopct='%1.1f%%',
    textprops={'color': 'blue'},
    pctdistance=0.8,
    wedgeprops={'linewidth': 8},
    colors=custom_colors 
)

plt.title("'Percentage of Each Image Type per Data Set'")
plt.axis('equal')

plt.legend(
    image_counts_summary['Set'],
    title='Data set folder',
    loc='upper left',
    bbox_to_anchor=(1, 0, 0.5, 1),
    handleheight=2
)

plt.savefig(f'{file_path}/labels_distribution_pie.png', bbox_inches='tight', dpi=150)
plt.show()


In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize=(8, 5))

custom_palette = ["#4287f5", "#f54242", "#42f56a"] 

sns.barplot(
    data=pd.DataFrame(image_counts, columns=['Set', 'Label', 'Frequency']),
    x='Set',
    y='Frequency',
    hue='Label',
    palette=custom_palette  
)

plt.savefig(f'{file_path}/labels_distribution.png', bbox_inches='tight', dpi=150)
plt.show()


### Enhancing the training process, by utilizing the ImageDataGenerator library to employ image data augmentation techniques. This will create a varied set of temporary images within the training dataset, thereby enhancing the training process

### Visualize Augmented Training Images##

In [ ]:
training_batch_size = 20

augmented_test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,                                                                            
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=training_batch_size,
    class_mode='binary',
    shuffle=False
)


augmented_train_set = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
).flow_from_directory(
    train_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=training_batch_size,
    class_mode='binary',
    shuffle=True
)


augmented_validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    validation_path,
    target_size=image_shape[:2],
    color_mode='rgb',
    batch_size=training_batch_size,
    class_mode='binary',
    shuffle=False
)

print("Test class Found {0} images belonging to {1} classes.".format(augmented_test_set.samples, len(augmented_test_set.class_indices)))

print("Train class Found {0} images belonging to {1} classes.".format(augmented_train_set.samples, len(augmented_train_set.class_indices)))

print("Validation class Found {0} images belonging to {1} classes.".format(augmented_validation_set.samples, len(augmented_validation_set.class_indices)))

# #---------------------------------------------


# training_batch_size = 20
# image_size_reduction = 0.75  # 25% reduction in size

# augmented_test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
#     test_path,
#     target_size=(int(image_shape[0] * image_size_reduction), int(image_shape[1] * image_size_reduction)),
#     color_mode='rgb',
#     batch_size=training_batch_size,
#     class_mode='binary',
#     shuffle=False
# )

# augmented_train_set = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='nearest'
# ).flow_from_directory(
#     train_path,
#     target_size=(int(image_shape[0] * image_size_reduction), int(image_shape[1] * image_size_reduction)),
#     color_mode='rgb',
#     batch_size=training_batch_size,
#     class_mode='binary',
#     shuffle=True
# )

# augmented_validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
#     validation_path,
#     target_size=(int(image_shape[0] * image_size_reduction), int(image_shape[1] * image_size_reduction)),
#     color_mode='rgb',
#     batch_size=training_batch_size,
#     class_mode='binary',
#     shuffle=False
# )

# print("Test class Found {0} images belonging to {1} classes.".format(augmented_test_set.samples, len(augmented_test_set.class_indices)))
# print("Train class Found {0} images belonging to {1} classes.".format(augmented_train_set.samples, len(augmented_train_set.class_indices)))
# print("Validation class Found {0} images belonging to {1} classes.".format(augmented_validation_set.samples, len(augmented_validation_set.class_indices)))


In [ ]:
datasets = {
    'Augmented Test Image': augmented_test_set,
    'Augmented Training Image': augmented_train_set,
    'Augmented Validation Image': augmented_validation_set
}

for dataset_name, dataset in datasets.items():
    for _ in range(1):
        img, _ = dataset.next()
        plt.imshow(img[0])
        plt.title(dataset_name)
        plt.show()


In [ ]:
joblib.dump(value=augmented_train_set.class_indices,
            filename=f"{file_path}/train_classes_indices.pkl")

# Creating the model section:

#### - Creating an image classification model using convolutional, pooling, dense, and dropout layers, compiles it, and returns the model. Summary() method is then called to display the model architecture summary.

In [ ]:
def create_image_classification_model():
    model = Sequential([
        Conv2D(32, (3, 3), input_shape=image_shape, activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
create_image_classification_model().summary()


# ----------------------------


- Training an image classification model using 'augmented_train_set' dataset and validating it using the 'augmented_validation_set' dataset. EarlyStopping callback to stop training if the validation accuracy does not improve for _5_ consecutive epochs.

In [ ]:
model = create_image_classification_model()

model.fit(augmented_train_set,
          epochs=15,
          validation_data=augmented_validation_set,
          verbose=1)

model.save('outputs/v2/model training/cherry_mildew_model.h5')
print(f"Model saved as {model}")

#----------------------------------------------



# Checking the performance of the created model

- Generating two plots: one for the loss values and one for the accuracy values from the model training history. Print plots as images.

In [ ]:
losses = pd.DataFrame(model.history.history)

sns.set_style("darkgrid")
losses[['loss', 'val_loss']].plot(style='.-', color=['#4287f5', '#f54242'])
plt.title("Loss")
plt.legend(['Training Loss', 'Validation Loss'])  # Change legend labels
plt.savefig(f'{file_path}/model_training_losses.png', bbox_inches='tight', dpi=150)
plt.show()

print("\n")

losses[['accuracy', 'val_accuracy']].plot(style='.-', color=['#42f56a', '#f5ad42'])
plt.title("Accuracy")
plt.legend(['Training Accuracy', 'Validation Accuracy'])  # Change legend labels
plt.savefig(f'{file_path}/model_training_acc.png', bbox_inches='tight', dpi=150)
plt.show()
## ---------------------------------------------



# Evaluating the created Model 

- 


In [ ]:
os.makedirs("outputs/v2/model training/evaluation", exist_ok=True)

model = load_model('outputs/v2/model training/cherry_mildew_model.h5')
loss_accuracy_evaluation = model.evaluate(augmented_test_set)
joblib.dump(value=loss_accuracy_evaluation, filename="outputs/v2/model training/evaluation/model_evaluation.pkl")

#---------------------------------------------

 # Prediction on new data
- Randomly selecting an image, perfoming classification prediction, and displaying the prediction classification, probability, and image file path to check.

In [ ]:
# Select a random label and image
random_label, random_image = random.choice([(label, image) for label in labels for image in os.listdir(test_path + '/' + label)])

# Full directory path of the randomly chosen image
image_path = os.path.join(test_path, random_label, random_image)

# Load and resize the image
pil_image = image.load_img(image_path, target_size=image_shape, color_mode='rgb')
my_image = np.expand_dims(image.img_to_array(pil_image) / 255.0, axis=0)

pred_proba = model.predict(my_image)[0, 0]

target_map = {v: k for k, v in augmented_train_set.class_indices.items()}
pred_class = target_map[pred_proba > 0.5]
if pred_class == target_map[0]:
    pred_proba = 1 - pred_proba

truncated_proba = math.trunc(pred_proba * 100 * 100) / 100
desired_path = '/'.join(image_path.split('/')[4:])

print("Prediction Classification:", pred_class)
print("Probability of Matching Classification: {}%".format(truncated_proba))
print("Image File Path check:", desired_path)
pil_image
#--------------------------


In [ ]:
%pip uninstall -y tensorflow keras pyarrow numpy scipy


In [ ]:
import os

def delete_directory(directory_path):
    """Deletes the specified directory and all of its contents.

    Args:
        directory_path (str): The path to the directory to delete.
    """

    if os.path.exists(directory_path):
        for file in os.listdir(directory_path):
            file_path = os.path.join(directory_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                delete_directory(file_path)
        os.rmdir(directory_path)

if __name__ == "__main__":
    # Get the path to the train and test directories.
    train_path = os.path.join(my_data_dir, "train")
    test_path = os.path.join(my_data_dir, "test")

    # Delete the train and test directories.
    delete_directory(train_path)
    delete_directory(test_path)

    print("Successfully deleted the train and test directories.")


# Summary

#### Objective: Respond to Business Requirement 2 by determining the health status (healthy or powdery mildew) of cherry leaves based on given images.

- Analyzed and visualized the distribution of images in the train, validation, and test sets.
- Prepared augmented image datasets for training, validation, and testing.
- Created a deep learning model for image classification using Convolutional Neural Networks (CNN).
- Trained the model on the augmented training set and evaluated its performance on the augmented validation set.
- Saved the trained model for future use.
- Plotted and analyzed the model's training loss and accuracy.
- Evaluated the model's performance on the augmented test set and saved the evaluation results.
- Made predictions on a randomly selected image using the trained model and displayed the prediction classification, probability, and image information.

### To Fix:
- Code for generating and displaying augmented images from the augmented_train_set is not working"- Fixed by correcting syntax
- loss and val_accuracy don't seem to be good as there is high loss and and low accuracy.
    - Tried increasing augmented data from 2 to 10 range to see if that improves accuracy but did not work.
        - Fixed by correcting the wrong selected columns. Corrected "['loss', 'val_accuracy']" to "['loss', 'val_loss']"
- Code wont run unless I have requirement.txt file ran at the beginning....
- Reduce dataset (Box 10) down from 7 each if possible.